<a href="https://colab.research.google.com/github/baeksumin/k-project/blob/main/data_preprocessing/TimeSeries_trend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, date, timedelta
from scipy import interpolate
from google.colab import drive

In [ ]:
# 조건에 해당하는 날짜 추출.
def get_point(result):
  x = list(result['x1'])+ list(result['x2'])
  x = list(set(x))
  x.sort()
  return x

# 전체 df에서 해당 날짜만 가져오기
def get_date(date_list):
  global scode1
  
  if(len(date_list)==0):   #아예 조건에 해당하는 점이 없을 경우
    return pd.DataFrame() 
    
  check_df =  scode1[scode1.x == date_list[0]]

  for i in date_list :
    df = scode1[scode1.x == i]
    check_df = check_df.append(df,ignore_index = True)

  check_df = check_df.iloc[1:,:]
  return check_df


# 두 점 사이 관계 df 
def two_point (check_df):

  df = check_df[['x']]
  df= df.iloc[:-1]

  df[['x2','y1','y2','t','p','m']] = np.nan
  df.rename(columns ={'x':'x1'}, inplace = True)

  for i in range(len(df)): 
    df.iloc[i,1] = check_df.iloc[i+1,0]
    df.iloc[i,2] = check_df.iloc[i,1]
    df.iloc[i,3] = check_df.iloc[i+1,1]

  for i in range(df.shape[0]):

    t = df.iloc[i,1] - df.iloc[i,0]
    df.iloc[i,4] = t.days

    y1 = df.iloc[i,2]
    y2 = df.iloc[i,3]
    result = (abs(y2 - y1 )) / ((y1+y2)/2 )
    df.iloc[i,5] = result

    m = df.iloc[i,3]- df.iloc[i,2]  
    if (m>0):
      df.iloc[i,6] = 1
    elif (m<0):
      df.iloc[i,6] = -1
    else :
      df.iloc[i,6] = 0

  return df


# 조건 필터링 
def p_t(df):
  t = df['t'] <5
  p = df['p']<0.05

  result = df[~t&~p]
  return result


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data_path = '/content/drive/MyDrive/[Shared]K-Project/data/'

# 저장된 개별종목 데이터 가져오기
with open(data_path + 'initial_data1.json',"r") as json_file:
    json_data = json.load(json_file)

#저장된 fullcode 데이터 가져오기
with open(data_path + 'full_code.pkl',"rb") as f: 
    fullcode = pickle.load(f)
    
# 상장폐지 상태 제외시키기
fullcode_filter = fullcode[fullcode['status'] != '상장폐지']

# fullcode열을 list로 변경
fullcode_list = list(fullcode_filter['full_code'])
print(fullcode_list[:10])

['KR7060310000', 'KR7095570008', 'KR7006840003', 'KR7054620000', 'KR7265520007', 'KR7211270004', 'KR7027410000', 'KR7282330000', 'KR7138930003', 'KR7001460005']


In [ ]:
result_dict={}
global scode1

num = 0
for j in fullcode_list:

  df = pd.read_json(json_data[j], orient ='index')
  scode = df.transpose()
  num = fullcode_list.index(j)
  print(num,"번째,  ", j)
   
  #시간 순 재정렬.
  scode2 = scode.sort_values(by=['TRD_DD'])
  scode2.reset_index(drop=True,inplace=True)
  scode2['TRD_DD']=pd.to_datetime(scode2['TRD_DD']) #datetime변환

  #시가총액 str->float 데이터타입변환
  scode2['MKTCAP'] = scode2['MKTCAP'].str.replace(',','').astype('float')

  #날짜, 시가총액 열만 추출
  scode1 = scode2[['TRD_DD','MKTCAP']]
  scode1 = scode1.rename(columns = {'TRD_DD':'x','MKTCAP':'y'})
  scode1.reset_index(drop=True,inplace=True)


  #기울기 변하는 지점 찾아주기
  ischange = list()

  for i in range(1,len(scode1)-2):
    m1 = scode1.iloc[i,1] - scode1.iloc[i-1,1]
    m2 = scode1.iloc[i+1,1] - scode1.iloc[i,1]

    if(m1*m2<=0):
        ischange.append(scode1.iloc[i,0])

  # 전체 df에서 기울기 변하는 지점들만 추출한 후 , (t=5, p=0.05) 에 해당하는 날짜  추출.

  check_df = get_date(ischange) #전체 df에서 기울기 변하는 날짜만 추출하기.
  if(len(check_df)==0):

    scode['TREND']= np.nan
    result_js = scode.to_json(orient = 'columns')

    result_dict[j] = result_js
    print("\n기울기 변하는 날짜가 존재 하지 않음. 조건 성립 X")
    print("-----------------------------------------------------------------------------------------\n")
    
    continue
  df = two_point(check_df)
  result = p_t(df)   # (t=5, p=0.05) 에 해당하는 날짜  추출.



  # 조건에 해당하는 날짜들끼리 다시 (t=5, p=0.05) 에 해당하는 날짜  추출. 
  red_x = get_point(result)
  red = get_date(red_x)

  if(len(red)==0): #아예 trend 점 안만들어지는 종목 에러 방지.
    scode['TREND']= np.nan
    result_js = scode.to_json(orient = 'columns')

    result_dict[j] = result_js
    print("조건에 해당하는 점이 아예 없음.")
    print("\n-----------------------------------------------------------------------------------------")
    continue

  df1=two_point(red)
  result2=p_t(df1)


  #점들이 모두 이어지고, 기울기가 계속 변하는 모습 나올때까지 반복 작업.
  while True:
      
    count = 0     
    red_x1 = get_point(result2) #조건에 만족하는 날짜 추출

    for i in range(result2.shape[0]-1):
      a = result2.iloc[i,1] == result2.iloc[i+1,0]
      b = result2.iloc[i,6]* result2.iloc[i+1,6]== -1

      if(a&b ) :
        count+=1
        
    if (count ==  result2.shape[0]-1 ):
        print(j+" 조건 성립 완료 \n")
        break
          
    for i in range(result2.shape[0]-1):

       #i번째 기울기 음수일때
      if (result2.iloc[i,6]== -1):  
        #i+1번째 기울기 양수일 때
        if (result2.iloc[i+1,6] == 1): 
          #점이 이어져 있지 않으면
          if (result2.iloc[i,1] != result2.iloc[i+1,0]):
            if(result2.iloc[i,3]> result2.iloc[i+1,2]):
               red_x1.remove(result2.iloc[i,1])
            else :
              red_x1.remove(result2.iloc[i+1,0]) 
                
        #i+1번째 기울기 음수일 때
        elif (result2.iloc [i+1,6]== -1):       
           red_x1.remove(result2.iloc[i,1])


      #i번째 기울기 양수일때
      else :   
        #i+1번째 기울기 양수일 때
        if (result2.iloc[i+1,6] == 1): 
          #점이 이어져 있지 않으면
          if (result2.iloc[i,1] != result2.iloc[i+1,0]):
            red_x1.remove(result2.iloc[i,1])
            red_x1.remove(result2.iloc[i+1,0]) 
          #점이 이어져 있으면
          else :
            red_x1.remove(result2.iloc[i,1])
        #i+1번째 기울기 음수일 때
        else :
          #점이 이어져 있지 않으면
          if (result2.iloc[i,1] != result2.iloc[i+1,0]):
            if (result2.iloc[i,3]>=result2.iloc[i+1,2]):
               red_x1.remove(result2.iloc[i+1,0])
            else:
              red_x1.remove(result2.iloc[i,1])
        
    final = get_date(red_x1)
    df1=two_point(final)
    result2=p_t(df1)


  # trend -1~1 사이 값으로 변환.

  final = get_date(red_x1) #최종 기울기 변하는 점 추출.

  for k in range(final.shape[0]-1): #기울기 변하는 곳 1, -1로 값 채워주기
    if(result2.iloc[k,6]== 1):
      final.iloc[k,1] = -1
    else :
      final.iloc[k,1] = 1

  # 마지막 끝 점 (-1,1)해당하는 값으로 채워주기
  n = final.shape[0]-2
  if(final.iloc[n,1]== -1):
     final.iloc[final.shape[0]-1,1] = 1
  else:
     final.iloc[final.shape[0]-1,1] = -1

   # -1~ 1 사이 점 채워주기.(linear interpolation)
  scode_trend = scode1[['x']]
  scode_trend['TREND'] = np.nan

  for i in range(len(final)):
    scode_trend.loc[scode_trend['x']== final.iloc[i,0],'TREND'] = final.iloc[i,1]

  scode_trend =  scode_trend.set_index('x')
  scode_trend = scode_trend[final.iloc[0,0]:final.iloc[len(final)-1,0]].interpolate(method = "time")


  # 마지막으로 원래 데이터에 TREND 열 만들어주어서 합치기.
  
  scode_trend.reset_index(inplace=True)
  scode_trend = scode_trend.rename(columns = {'x':'TRD_DD'})
  scode['TRD_DD']=pd.to_datetime(scode['TRD_DD'])
  scode = pd.merge(scode, scode_trend, on='TRD_DD', how='left')
  scode['TRD_DD'] = scode['TRD_DD'].astype(str).str.replace('-','/')
  

  result_js = scode.to_json(orient = 'columns') # 데이터프레임을 json 형태로 변환
  result_dict[j] = result_js # fullcode가 key값이 되고 json이 value값이 됨

  print("-----------------------------------------------------------------------------------------")


0 번째,   KR7060310000
KR7060310000 조건 성립 완료 

-----------------------------------------------------------------------------------------
1 번째,   KR7095570008
KR7095570008 조건 성립 완료 

-----------------------------------------------------------------------------------------
2 번째,   KR7006840003
KR7006840003 조건 성립 완료 

-----------------------------------------------------------------------------------------
3 번째,   KR7054620000
KR7054620000 조건 성립 완료 

-----------------------------------------------------------------------------------------
4 번째,   KR7265520007
KR7265520007 조건 성립 완료 

-----------------------------------------------------------------------------------------
5 번째,   KR7211270004
KR7211270004 조건 성립 완료 

-----------------------------------------------------------------------------------------
6 번째,   KR7027410000
KR7027410000 조건 성립 완료 

-----------------------------------------------------------------------------------------
7 번째,   KR7282330000
KR7282330000 조건 성립 완료 

----------

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:659: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[k] = np.nan


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
KR7270660004 조건 성립 완료 

-----------------------------------------------------------------------------------------
1353 번째,   KR7038680005
KR7038680005 조건 성립 완료 

-----------------------------------------------------------------------------------------
1354 번째,   KR7137310009
KR7137310009 조건 성립 완료 

-----------------------------------------------------------------------------------------
1355 번째,   KR7217480003
KR7217480003 조건 성립 완료 

-----------------------------------------------------------------------------------------
1356 번째,   KR7121890008
KR7121890008 조건 성립 완료 

-----------------------------------------------------------------------------------------
1357 번째,   KR7097780001
KR7097780001 조건 성립 완료 

-----------------------------------------------------------------------------------------
1358 번째,   KR7042110007
KR7042110007 조건 성립 완료 

-----------------------------------------------------------------------------------------
1359 번째,   KR729806000

In [ ]:
#데이터 저장
with open(data_path + 'trend.json', 'w') as f:
  json.dump(result_dict, f)